## Estimation of potential evapotranspiration (PET) based on differnet methods using the Python package *PyEt*  
This script estimates potential evapotranspiration (PET) using 18 different methods in the Python package PyEt.
It uses several meteorological parameters and needs the following parameter time series (with daily values): 

- mean temperature (tmean.xlsx)
- max temperature (tmax.xlsx)
- min temperature (tmin.xlsx)
- mean relative humidity (rh.xlsx)
- solar radiation [MJ m^(-2) d^(−1)] (rs.xlsx)
- mean day wind speed (wind.xlsx)
- Latitude
- Elevation

To simplify the use of this script, place each parameter time series in separate xlsx-files in 1_input_data\PET_data and call them the name in parenthesis.

Source: *https://pyet.readthedocs.io/en/latest/index.html*  

Joaquim Altimiras Granel, 2024

In [4]:
# Import libraries

import pandas as pd
import pyet
import numpy
import pyet
import os

# Create an empty DataFrame to store the potential evapotranspiration (PE) results for different methods
pet_results = pd.DataFrame()

# Load each parameter rename each column to match the expected parameter name
tmean_data = pd.read_excel(r"1_input_data\PET_data\tmean.xlsx", index_col=0, parse_dates=True)
tmean_data.rename(columns={tmean_data.columns[0]: 'tmean'}, inplace=True)

rh_data = pd.read_excel(r"1_input_data\PET_data\rh.xlsx", index_col=0, parse_dates=True)
rh_data.rename(columns={rh_data.columns[0]: 'rh'}, inplace=True)

rs_data = pd.read_excel(r"1_input_data\PET_data\rs.xlsx", index_col=0, parse_dates=True)
rs_data.rename(columns={rs_data.columns[0]: 'rs'}, inplace=True)

tmax_data = pd.read_excel(r"1_input_data\PET_data\tmax.xlsx", index_col=0, parse_dates=True)
tmax_data.rename(columns={tmax_data.columns[0]: 'tmax'}, inplace=True)

tmin_data = pd.read_excel(r"1_input_data\PET_data\tmin.xlsx", index_col=0, parse_dates=True)
tmin_data.rename(columns={tmin_data.columns[0]: 'tmin'}, inplace=True)

wind_data = pd.read_excel(r"1_input_data\PET_data\wind.xlsx", index_col=0, parse_dates=True)
wind_data.rename(columns={wind_data.columns[0]: 'wind'}, inplace=True)

# Define the latitude and elevation separately as single values
lat_value = input("Enter the latitude of the location in decimal degrees: ")
lat = float(lat_value) * numpy.pi / 180
elevation_value = input("Enter the elevation of the location in meters above sea level: ")
elevation = float(elevation_value)

# Align Datasets
# Combine all datasets into one DataFrame, aligning them based on datetime index
combined_data = pd.concat([tmean_data, wind_data, rs_data, rh_data, tmax_data, tmin_data], axis=1, join="outer")

# Calculate Potential Evapotranspiration (PE) using pyet
pet_results = pyet.calculate_all(tmean=combined_data["tmean"], wind=combined_data["wind"],
                           rs=combined_data["rs"], elevation=elevation,
                           lat=lat, tmax=combined_data["tmax"],
                           tmin=combined_data["tmin"], rh=combined_data["rh"])

# Reset index setting
pet_results = pet_results.reset_index()

# Save the PET DataFrame to a xlsx file
# Input name of original file
original_file_name = input("Enter the name of the file to save the PE results: ")

# Create a directory to store the output file
output_dir = r'2_data_processing\2.3_PET_data'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Generate the file name for the new Excel file
base_file_name = os.path.basename(original_file_name)
new_file_name = 'pet_results_' + base_file_name + '.xlsx'
file_path = os.path.join(output_dir, new_file_name)

# Export DataFrame to Excel
try:
    pet_results.to_excel(file_path, index=False)
    print(f"Data successfully exported to {file_path}")
except Exception as e:
    print(f"Error exporting data: {e}")


# Print a sample of the calculated PE DataFrame
print(pet_results.head())

Data successfully exported to C:\Users\JMAG\OneDrive - Chalmers\Arbetsfiler Joaquim\Working material\4_outputs\potential_evapotranspiration\pet_results_DF164.xlsx
